# Test - self reconstruction

In [1]:
import torch
from torch.autograd import Variable
import numpy as np
from utils import Hps
from utils import DataLoader
from solver import Solver
import h5py

In [4]:
def calc_loss(sp_out, sp_trg):
    min_idx = min(len(sp_out), len(sp_trg)) - 1
    return np.mean(np.abs(sp_out[0:min_idx] - sp_trg[0:min_idx]))

def convert_sp(sp, c, solver, gen=True):
    c_var = Variable(torch.from_numpy(np.array([c]))).cuda()
    sp_tensor = torch.from_numpy(np.expand_dims(sp, axis=0))
    sp_tensor = sp_tensor.type(torch.FloatTensor)
    converted_sp = solver.test_step(sp_tensor, c_var, gen=gen)
    converted_sp = converted_sp.squeeze(axis=0).transpose((1, 0))
    return converted_sp

def get_model(hps_path, model_path, wavenet_mel):
    hps = Hps()
    hps.load(hps_path)
    hps_tuple = hps.get_tuple()
    solver = Solver(hps_tuple, None, wavenet_mel)
    solver.load_model(model_path)
    return solver

def get_speakers():
    with open(speaker_used_path) as f:
        speakers = [line.strip() for line in f]
        speaker2id = {speaker:i for i, speaker in enumerate(speakers)}
    return speakers, speaker2id

def test(speaker, speaker2id, solver, gen, f_h5, f_trg_h5, decompression):
    dset='test'
    losses=[]
    for utt_id in f_h5[f'{dset}/{speaker}']:
        sp = f_h5[f'{dset}/{speaker}/{utt_id}'][()]
        sp_trg = f_trg_h5[f'{dset}/{speaker}/{utt_id}'][()]
        sp_in = sp if decompression else sp_trg
        sp_out = convert_sp(sp_in, speaker2id[speaker], solver, gen=gen)
        loss = calc_loss(sp_out, sp_trg)
        losses.append(loss)
    return losses

def test_all(h5_path, h5_trg_path, speaker_used_path, model_path, hps_path, wavenet_mel, decompression, gen=False):
    loss_sum=0
    samples_couter=0
    speakers, speaker2id = get_speakers()
    solver = get_model(hps_path, model_path, wavenet_mel)
    
    with h5py.File(h5_path, 'r') as f_h5:
        with h5py.File(h5_trg_path, 'r') as f_trg_h5:
            for speaker in speakers:
                losses = test(speaker, speaker2id, solver, gen, f_h5, f_trg_h5, decompression)
                loss_sum += np.sum(losses)
                samples_couter += 1
    return loss_sum/samples_couter

In [8]:
h5_trg_path = "vctk_old/data-wavenet.h5"
h5_path = "vctk_old/data-8-wavenet.h5"
model_path = "model_vctk/mel/model_vctk-90000"
speaker_used_path = "vctk_old/en_speaker_used.txt"
hps_path="/tf/notebooks/SKAJPAI/voice_style_transfer/implementation/vctk.json"
decompression=True
wavenet_mel=True

mean_loss = test_all(h5_path, h5_trg_path, speaker_used_path, model_path, hps_path, wavenet_mel, decompression)
print('mean loss: ',mean_loss)

load model from model_vctk/mel/model_vctk-90000
mean loss:  1.4487096190452575
